# Segmenting and clustering neighborhoods in Toronto 
*Author: Xinyue Luo*
### Part 1

In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests 
import pandas as pd

### Scrape list of post codes of Canada from wikipedia page

In [ ]:
 web_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(web_url,'lxml')
print(soup.prettify())

### Get table components marked with class "wikitable sortable"

In [3]:
table = soup.find('table', class_='wikitable sortable')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

### Extract header and rows of the table into lists, then convert into pandas dataframe

In [4]:
# get headers as columns
headers = table.find_all('th')

column_names = [header.text for header in headers]

# remove '\n' in the 'Neighbourhood' column
column_names = [header.replace('\n','') for header in column_names]
column_names

['Postcode', 'Borough', 'Neighbourhood']

In [5]:
# get rows
rows = table.find_all('tr')
len(rows)

290

In [6]:
# get data in rows and fill into a list
table_list = []
for row in rows:
    tds = row.find_all('td')
    row_list = [td.text for td in tds]
    # remove '\n' in last column
    row_list = [td.replace('\n', '') for td in row_list]
    table_list.append(row_list)

# display table in list    
table_list

[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M

In [7]:
# convert to pandas dataframe
df_toronto = pd.DataFrame(table_list[1:], columns=column_names)
print(df_toronto.shape)
df_toronto.head(10)

(289, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


## Data cleanups
### Step 1. remove entries with "Not assigned" boroughs 
### Step 2. replace "Not assigned" neighbourhoods with borough name
### Step 3. combine neighbourhoods with the same postcode into one row, seperate neighborhood name by comma

In [8]:
# remove rows with Borough 'Not assigned'
df_toronto = df_toronto[~df_toronto.Borough.str.contains("Not assigned")]
print(df_toronto.shape)

# 
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head(10)

(212, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [9]:
# replace 'not assigned' neighbourhoods with borough
for index, row in df_toronto.iterrows():
    if row['Neighbourhood'] == "Not assigned":
        row['Neighbourhood'] = row['Borough']
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [10]:
# combine neighbourhoods with the same postcode
df = df_toronto.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda col: ', '.join(col))
df = pd.DataFrame(df)
df.reset_index(inplace=True)
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Display shape of final table after cleanups

In [13]:
# save to csv; print final shape of cleaned table
df.to_csv('toronto_1.csv', index=False)
print(df.shape)

(103, 3)
